In [33]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import pyrtools as pt
import plenoptic as po
import scipy.io as sio
import h5py
torch.__version__
numpy = lambda x: x.detach().cpu().numpy().squeeze()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

parameterTypes = ['pixel_statistics','mean_magnitude_pyramid_bands', 'ace','skew','kurt','acr','C0','Cx0','Cr0','Crx0','varHPR']
torch.set_default_dtype(torch.float64) 

In [49]:
## Comparison between python output and matlab output
for n in range(0,5):
    f = h5py.File(f'/Users/kathrynbonnen/Documents/MATLAB/textureSynth/pythonTestVectors/PortillaSimoncelliMatlab{n}.mat','r')
    # load in relevant variables from matlab
    paramsVector = f['paramsVector'][()][0]
    paramIndices = [0]+list(f['paramIndices'][()][0])
    im0 = torch.Tensor(f['im0'][()]).T.unsqueeze(0).unsqueeze(0);
    Nsc = int(f['Nsc'][()][0][0])
    Na = int(f['Na'][()][0][0])
    Nor = int(f['Nor'][()][0][0])
    
    # matlab PS output
    y_mat = torch.Tensor(paramsVector)

    # calculate python output
    model_py = po.simulate.Portilla_Simoncelli(im0.shape[-2:],n_scales=Nsc,n_orientations=Nor,Na=3)
    y_py = model_py.forward(im0)

    # Find parameters above the tolerance
    atol = 1e-5
    rtol = 1e-4
#     np.testing.assert_allclose(y_py[:], y_mat[:], rtol=rtol, atol=atol)
    
    abovTol = np.nonzero(~np.isclose(y_py[:], y_mat[:], rtol=rtol, atol=atol))
    print(abovTol)
    print(y_py[abovTol])
    print(y_mat[abovTol])
    

    

    

(array([], dtype=int64),)
tensor([])
tensor([])
(array([], dtype=int64),)
tensor([])
tensor([])
(array([], dtype=int64),)
tensor([])
tensor([])
(array([], dtype=int64),)
tensor([])
tensor([])
(array([], dtype=int64),)
tensor([])
tensor([])


In [ ]:
aboveTol = np.nonzero(~np.isclose(y_py[:], y_mat[:], rtol=rtol, atol=atol))[0]


In [16]:
## Comparison between python output (with pyramid input) and matlab output

for n in range(0,5):
    f = h5py.File(f'/Users/kathrynbonnen/Documents/MATLAB/textureSynth/pythonTestVectors/PortillaSimoncelliMatlab{n}.mat','r')

    # load in relevant pyramid variables from matlab
    Nsc = np.int64(f['Nsc'][()][0][0]) # n scales
    Nor = np.int64(f['Nor'][()][0][0]) # n orientations
    im0 = torch.Tensor(f['im0'][()]).T.unsqueeze(0).unsqueeze(0) # image
    pyr0 = f['pyr0'][()].flatten() # pyramid
    paramsVector = f['paramsVector'][()][0] # matlab PS output
    paramIndices = [0]+list(f['paramIndices'][()][0])

    # convert matlab paramsVector to Tensor
    y_mat = torch.Tensor(paramsVector)

    # run forward model beginning with matlab pyramid
    matlab_pyr = po.simulate.Steerable_Pyramid_Freq.vector_to_pyramid(pyr0, im0.shape, Nsc, Nor)

    model = po.simulate.Portilla_Simoncelli(im0.shape[-2:],n_scales=4,n_orientations=4,Na=3,inputImageIsPyramid=True)
    y_py = model.forward((matlab_pyr,im0))
    atol = 1e-5
    rtol = 1e-5
    aboveTol = np.nonzero(~np.isclose(y_py[:], y_mat[:], rtol=rtol, atol=atol))[0]
    parameterTypesAboveTolerance = [value for ii,value in enumerate(parameterTypes) if 
                                    np.any(np.logical_and(aboveTol>=paramIndices[ii], aboveTol<paramIndices[ii+1]))]

    
    print(parameterTypesAboveTolerance)
    print(aboveTol.shape)
#     tmp = np.testing.assert_allclose(y[:], y_mat[:], rtol=rtol, atol=atol)

['acr', 'Cx0', 'Cr0', 'Crx0']
(117,)
['C0', 'Cr0', 'Crx0']
(69,)
['Cr0', 'Crx0']
(68,)
['C0', 'Cr0', 'Crx0']
(71,)
['Cx0', 'Cr0', 'Crx0']
(95,)
